# Fun With Types

We haven't used any of Julia's lovely [sophisticated type system](https://docs.julialang.org/en/latest/manual/types.html). Let's fix that now. I will show some code and there are undoubtedly *many* possible improvements, which we'll leave (for now) as an **exercise**. If you want, believe that I'm leaving things simple for clarity, rather than because I'm a simple person.

## Goals of this section
1. Meld our previous "matrix maker" functions with Julia types to make our code more flexible
2. Make things more convenient by extending methods in `Base`
3. Examine phase transitions in various physics models

In [ ]:
abstract type AbstractHamiltonian{T, S:<AbstractMatrix} <: AbstractArray{T,N} end
# Hamiltonians may be real or complex, but are always Hermitian

Base.ishermitian(A::AbstractHamiltonian) = true

type TransverseFieldIsing{Tv, S <: AbstractMatrix} <: AbstractHamiltonian{Tv, S}
    L::Int
    basis::Vector # may be a vector of BitVectors/BitMatrices/Vectors/Matrices
    mat::S #this may be sparse, or Diagonal, or something else
    h::Real
    function TransverseFieldIsing{Tv, S}(L::Integer, h::Real=0.) where Tv where S <: Matrix
        basis = generate_basis(L)
        H = zeros(2^L, 2^L)
        for (index, element) in enumerate(basis)
            # the diagonal part is easy
            diag_term = 0.
            for site in 1:L-1
                diag_term -= !xor(element[site], element[site+1])
            end
            H[index, index] = diag_term
            # off diagonal part
            for site in 1:L-1
                mask = falses(L)
                mask[site] = true
                new_element = xor.(element, mask)
                new_index = int_rep(new_element, L)
                H[index, new_index] = -h
            end
        end
        new(L, basis, Hermitian(H), h)
    end
    function TransverseFieldIsing{Tv, S}(L::Integer, h::Real=0.) where Tv where S <: SparseMatrix
    end
end

## Exercise

Write the constructor for the `TransverseFieldIsing` type that generates a *sparse* representation of the matrix. We'll need this to go to bigger system sizes. Some suggestions:

  1. Can we pre-allocate the arrays for the sparse matrix, since for this Hamiltonian we know *exactly* how many non-zero elements per row there could maximally be?
  2. Generating the basis separately is pretty slow - can we fully enumerate it using the Hamiltonian as we generate each row?
  3. It's probably a good idea to write some tests to make sure that our two representations match...

In [ ]:
# A little more fun with types
Base.eigfact(A::TransverseFieldIsing; kwargs...) = eigfact(A.Mat; kwargs...)
Base.eigvals(A::TransverseFieldIsing; kwargs...) = eigvals(A.Mat; kwargs...)
Base.eigvecs(A::TransverseFieldIsing; kwargs...) = eigvecs(A.Mat; kwargs...)
Base.eig(A::TransverseFieldIsing; kwargs...)     = eig(A.Mat; kwargs...)
Base.size(A::TransverseFieldIsing)               = size(A.Mat)
Base.size(A::TransverseFieldIsing, dim::Int)     = size(A.Mat, dim)
Base.ndims(A::TransverseFieldIsing)              = 2
Base.issparse(A::TransverseFieldIsing)           = issparse(A.Mat)

We have many similar models in physics, including the quantum XY, XXZ, and Heisenberg models. These are more physically realistic models for the behaviour of spins. Now, the spins are free to rotate and may not point in the $\hat{z}$ direction. This gives us a "richer" phase diagram and provides a better model of magnetism in real materials. Below, you'll find some more information about the models.

The *Heisenberg model* has the Hamiltonian:

$$ \hat{H}_{Heis} = -\sum_{\langle i, j \rangle} \hat{\vec{\sigma}}_i \cdot \hat{\vec{\sigma}}_j = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^x \hat{\sigma}_j^x + \hat{\sigma}_i^y \hat{\sigma}_j^y + \hat{\sigma}_i^z \hat{\sigma}_j^z $$

In the Heisenberg model, the spins would "like to" align in the same direction, but that direction could be any direction, and it *can move*. The overall magnetic moment may be rotating (as long as all the microscopic constituents are aligned). We say that the Heisenberg model has $SU(2)$ rotation symmetry (which is continuous) as opposed to the $\mathbb{Z}_2$ *spin flip* symmetry (which is discrete) of the Ising model with no transverse field. The different symmetries give rise to very different physics.

The $XXZ$ model introduces some *anisotropy* in the $\hat{z}$ direction, so that:

$$ \hat{H}_{XXZ} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^x \hat{\sigma}_j^x + \hat{\sigma}_i^y \hat{\sigma}_j^y +  \Delta\hat{\sigma}_i^z \hat{\sigma}_j^z $$

and finally, the $XY$ model sets $\Delta = 0$ so that:

$$ \hat{H}_{XY} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^x \hat{\sigma}_j^x + \hat{\sigma}_i^y \hat{\sigma}_j^y $$

The $XY$ model supports a very interesting phase transition, the description of which recently [won a Nobel prize](https://www.nobelprize.org/nobel_prizes/physics/laureates/2016/advanced-physicsprize2016.pdf).

In [ ]:
type Heisenberg{Tv, S <: AbstractMatrix} <: AbstractHamiltonian{Tv, S}
    L::Int
    basis::Vector
    Mat::S
    function Heisenberg{Tv, S}(L::Integer) where Tv where S <: Matrix
    end
    function Heisenberg{Tv, S}(L::Integer) where Tv where S <: SparseMatrix
    end
end

If you're following along at home and not pressed for time, feel free to try all of them, but if you're doing this with me let's focus on the XXZ model:

$$ \hat{H}_{XXZ} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^x \hat{\sigma}_j^x + \hat{\sigma}_i^y \hat{\sigma}_j^y +  \Delta\hat{\sigma}_i^z \hat{\sigma}_j^z $$

$\Delta$ is some real number between 0 and 1 that tunes the "anisotropy" of the system. To refresh our memories, the Pauli spin operators are:

In [1]:
σˣ = [0 1; 1 0]

2×2 Array{Int64,2}:
 0  1
 1  0

In [2]:
σʸ = [0 -im; im 0]

2×2 Array{Complex{Int64},2}:
 0+0im  0-1im
 0+1im  0+0im

In [3]:
σᶻ = [1 0; 0 -1]

2×2 Array{Int64,2}:
 1   0
 0  -1

If you're having trouble picturing how these *bond operators* (the pairs of $\sigma$s) should act, we can get Julia to help us:

In [5]:
kron(σˣ,σˣ)

4×4 Array{Int64,2}:
 0  0  0  1
 0  0  1  0
 0  1  0  0
 1  0  0  0

In [6]:
kron(σʸ,σʸ)

4×4 Array{Complex{Int64},2}:
  0+0im  0+0im  0+0im  -1+0im
  0+0im  0+0im  1+0im   0+0im
  0+0im  1+0im  0+0im   0+0im
 -1+0im  0+0im  0+0im   0+0im

We can actually make this much simpler to express in code using a little trick: rewrite in terms of the *raising and lowering operators* (also named *ladder operators*) $\sigma^+$ and $\sigma^-$:

In [7]:
σ⁺ = (σˣ + im*σʸ)/2

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im

In [9]:
σ⁺*[0,1]

2-element Array{Complex{Float64},1}:
 1.0+0.0im
 0.0+0.0im

In [10]:
σ⁺*[1,0]

2-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+0.0im

In [8]:
σ⁻ = (σˣ - im*σʸ)/2

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im

Which turns our Hamiltonian into (you can work this out by hand if you like):

$$ \hat{H}_{XXZ} = -\sum_{\langle i, j \rangle} 4\hat{\sigma}_i^+ \hat{\sigma}_j^- + 4\hat{\sigma}_i^- \hat{\sigma}_j^+ +  \Delta\hat{\sigma}_i^z \hat{\sigma}_j^z $$

Hopefully you can see why expressing it this way will be easier to implement with the bit-hacks. If you were doing everything with the matrix-vector ops this wouldn't matter but that doesn't scale well. Now you can turn this into a function to make the XXZ model.

## Exercise

Write a type and some constructors for each of the XXZ, XY, and Heisenberg models. Analogous to what we did for the Ising model, find the transition - does it look the same for different system sizes? You might be able to replicate some of your work by writing a few functions for "common" bond operations. The ladder operators will make this code much easier to write (hint: you do not need complex numbers).

**If you get stuck:**

Generically in a physics code *constructing* the Hamiltonian takes much less time than *solving* its eigensystem, so it's ok to do things in a "less optimal" way that is conceptually simpler for you, especially since no one has publications riding on this workshop. Some suggestions, if you don't know how to start:

1. Remember in the previous part, how we added up all the `kron`-ed together matrices? For each bond (pair of sites) you can construct the full $(2^L, 2^L)$ matrix for just that bond and then add up all the (sparse) matrices - for small systems, this is a good way to check your work.
2. We need to check if a bond is "flippable", that is, is one spin up and the other down? For traditional/obfuscatory/performance reasons this is often done in one line with `xor` *but* if you want to precompute a truth table OR write 4 `if-else` lines that is fine!
3. Also, if you're finding it hard to reason about how the bit and integer representations connect, it's fine to explicitly convert back and forth. You can go index by index, converting each to a bit representation (a `Vector{Bool}` might be less confusing), and seeing for each one what moves to other bit representations are possible. Then you can convert those bit representations back to integer representations, fill in the matrix, and continue.
4. You might find it most efficient to do something like:

```julia
function σ⁺σ⁻(input_config)
    
end

function σ⁻σ⁺(input_config)
    
end

function σᶻσᶻ(input_config)
    
end

function possible_moves(input_config_index, basis)
    input_config = basis[input_config_index]
    # diagonal term
    
    # off diagonal terms
    flipped_config_a = σ⁺σ⁻(input_config)
    flipped_config_b = σ⁻σ⁺(input_config)
    
    # you'll need to find the indices of these new configurations in the basis...
    index_a = convert_bit_rep_to_index(flipped_config_a)
    H[input_config_index, index_a] = ...
end
```

**Note** if this is taking too long/you want to see more of the physics but don't have time to write code for all the models, split off into teams of 2-3! One person can do XXZ, one can do Heisenberg, etc.

In [ ]:
type XXZ{Tv, S <: AbstractMatrix} <: AbstractHamiltonian{Tv, S}
    L::Int
    basis::Vector
    Mat::S
    Δ::Tv
    function XXZ{Tv, S}(L::Integer, Δ::Tv) where Tv where S <: Matrix
    end
    function XXZ{Tv, S}(L::Integer, Δ::Tv) where Tv where S <: SparseMatrix
    end
end

We should make sure that our models are working before anything else.
You'll probably want to extend the eigenfactorization functions for these models the same way we did for `TransverseFieldIsing`.

### Exercise
Check that if you set $\Delta$ large (small) enough, the $XXZ$ model has non-zero (zero) magnetization (you can reuse your code from the previous parts). Note that $\Delta = 0$ corresponds to the $XY$ model. Make sure that when $\Delta = 0$ ($XY$) or when $\Delta = 1$ (Heisenberg model) your energies for all three match up.